<a href="https://colab.research.google.com/github/yuyangweng/NLP/blob/main/NLP_RNN_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# # 6-1 加 embedding layer

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

# RNN寫文章

首先，我們預處理數據：

In [ ]:
import tensorflow
from tensorflow import keras
from tensorflow.keras.layers import SimpleRNN
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
INPUT_FILE = "/content/drive/My Drive/00NLP/alice_in_wonderland.txt"

# extract the input as a stream of characters
print("Extracting text from input...")
fin = open(INPUT_FILE, 'rb')
lines = []
for line in fin:
    line = line.strip().lower()  #刪除空白或結尾符號
    line = line.decode("ascii", "ignore")
    if len(line) == 0:
        continue
    lines.append(line)
fin.close()
text = " ".join(lines) #連成 1 string

Extracting text from input...


In [ ]:
# creating lookup tables
# Here chars is the number of features in our character "vocabulary"
chars = set([c for c in text]) #不重複字母或符號
nb_chars = len(chars)
char2index = dict((c, i) for i, c in enumerate(chars))
index2char = dict((i, c) for i, c in enumerate(chars))

In [ ]:
# create inputs and labels from the text. We do this by stepping
# through the text ${step} character at a time, and extracting a 
# sequence of size ${seqlen} and the next output char. For example,
# assuming an input text "The sky was falling", we would get the 
# following sequence of input_chars and label_chars (first 5 only)
#   The sky wa -> s
#   he sky was ->  
#   e sky was  -> f
#    sky was f -> a
#   sky was fa -> l
print("Creating input and label text...")
SEQLEN = 10
gap = 1

input_chars = []
label_chars = []
for i in range(0, len(text) - SEQLEN, gap):
    input_chars.append(text[i:i + SEQLEN]) #每筆輸入10個連續字母或空格
    label_chars.append(text[i + SEQLEN]) #對應輸出1個字母或空格

Creating input and label text...


In [ ]:
# vectorize the input and label chars
# Each row of the input is represented by seqlen characters, each 
# represented as a 1-hot encoding of size len(char). There are 
# len(input_chars) such rows, so shape(X) is (len(input_chars), seqlen, nb_chars).
# 對應(batch_size，timesteps，input_features)輸入
#
# Each row of output is a single character, also represented as a
# dense encoding of size len(char). Hence shape(y) is (len(input_chars),
# nb_chars).
print("Vectorizing input and label text...")
# X = np.zeros((len(input_chars), SEQLEN, nb_chars), dtype=np.bool)
X = np.zeros((len(input_chars), SEQLEN))
dataY = []
for i, input_char in enumerate(input_chars):#全部10個連續字母或空格之list
    for j, ch in enumerate(input_char):#單1組10個連續字母或空格之string
        # X[i, j, char2index[ch]] = 1
        X[i, j] = char2index[ch] # label encoding
    dataY.append(char2index[label_chars[i]])
y = utils.to_categorical(np.array(dataY))

Vectorizing input and label text...


In [ ]:
# Build the model. We use a single RNN with a fully connected layer
# to compute the most likely predicted output char
HIDDEN_SIZE = 256
BATCH_SIZE = 128
NUM_ITERATIONS = 25
NUM_EPOCHS_PER_ITERATION = 1
NUM_PREDS_PER_EPOCH = 100
 
model = Sequential()
model.add(????




# model.add(SimpleRNN(HIDDEN_SIZE, return_sequences=False,
#                     input_shape=(SEQLEN, nb_chars)))
model.add(Dense(nb_chars))
model.add(Activation("softmax"))
 
model.compile(loss="categorical_crossentropy", optimizer="adam")

In [ ]:
# We train the model in batches and test output generated at each step
for iteration in range(NUM_ITERATIONS):
    print("=" * 50)
    print("Iteration #: %d" % (iteration))
    model.fit(X, y, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS_PER_ITERATION)
    
    # testing model
    # randomly choose a string from input_chars[list of strings], then use it to 
    # generate text from model for next 100 chars
    test_idx = np.random.randint(len(input_chars))
    test_chars = input_chars[test_idx]
    print("Generating from seed: %s" % (test_chars))
    print(test_chars, end="")
    for i in range(NUM_PREDS_PER_EPOCH):
 
        Xtest = np.zeros((1, SEQLEN, nb_chars))
        for i, ch in enumerate(test_chars):
            # Xtest[0, i, char2index[ch]] = 1
            Xtest ???


        pred = model.predict(Xtest, verbose=0)
        ypred = index2char[np.argmax(pred)]
        print(ypred, end="")
        # move forward with test_chars + ypred
        test_chars = test_chars[1:] + ypred
    print()